In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [5]:
device = torch.device('cpu')

In [12]:
EPOCHS = 90
BATCH_SIZE = 128
MOMENTUM = 0.9
LR = 0.1
LR_DECAY = 0.0005
NUM_CLASSES = 1000
TRAIN_IMG_DIR = 'imagenet'
IMG_DIM = 227


In [13]:
class AlexNet(nn.Module):
  def __init__(self, num_classes=1000):
    super().__init__()

    self.net = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),
      nn.ReLU(),
      nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2),
      nn.MaxPool2d(kernel_size=3, stride=2),
      nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2),
      nn.ReLU(),
      nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2),
      nn.MaxPool2d(kernel_size=3, stride=2),
      nn.Conv2d(256, 384, 3, padding=1),
      nn.ReLU(),
      nn.Conv2d(384, 384, 3, padding=1),
      nn.ReLU(),
      nn.Conv2d(384, 256, 3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=3, stride=2)
    )

    self.classifier = nn.Sequential(
      nn.Dropout(p=0.5, inplace=True),
      nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
      nn.ReLU(),
      nn.Dropout(p=0.5, inplace=True),
      nn.Linear(in_features=4096, out_features=4096),
      nn.ReLU(),
      nn.Linear(in_features=4096, out_features=num_classes),
    )
    self.init_bias()

  def init_bias(self):
    for layer in self.net:
      if isinstance(layer, nn.Conv2d):
        nn.init.normal_(layer.weight, mean=0, std=0.01)
        nn.init.constant_(layer.bias, 0)
    nn.init.constant_(self.net[4].bias, 1)
    nn.init.constant_(self.net[10].bias, 1)
    nn.init.constant_(self.net[12].bias, 1)

  def forward(self, x):
    x = self.net(x)
    x = x.view(-1, 256*6*6)
    return self.classifier(x)

In [14]:
if __name__ == '__main__':
  seed = torch.manual_seed(42)

  alexnet = AlexNet().to(device)
  print(alexnet)
  print('AlexNet Created!')

  dataset = datasets.ImageFolder(TRAIN_IMG_DIR, transforms.Compose([
    transforms.CenterCrop(IMG_DIM),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ]))
  print('Dataset Created!')

  dataloader = data.DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True,
    num_workers=8,
    drop_last=True,
  )
  print('DataLoader Created!')

  optimizer = optim.Adam(alexnet.parameters(), lr=LR)
  print('Optimizer Created!')

  lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
  print('LR Scheduler Created!')

  for epoch in range(EPOCHS):
    for imgs, classes in dataloader:
      imgs, classes = imgs.to(device), classes.to(device)
      output = alexnet(imgs)
      print(classes, output)
      loss = F.cross_entropy(output, classes)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      print(loss)

    torch.save(alexnet.state_dict(), f'alexnet-{epoch}.pth')

AlexNet(
  (net): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): ReLU()
    (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=True)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReL

FileNotFoundError: [Errno 2] No such file or directory: 'imagenet'

In [14]:
F.cross_entropy(torch.tensor([0, 10]).float(), torch.tensor([0, 10]).float())

tensor(0.0005)